In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib as mpl
from matplotlib import cm
from sklearn.linear_model import LinearRegression
import numpy as np
import statsmodels.api as sm
from sklearn.datasets import make_blobs
import seaborn as sns
plt.style.use('seaborn')

In [31]:
df = pd.read_stata('Staat vd Stad.dta')
pd.set_option('display.max_columns', None)

C:\Users\jabor\AppData\Roaming\Python\Python38\site-packages\pandas\io\stata.py:1433: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)


In [33]:
df['V4']=df['V4'].map({'rarely ever or never': '2-low', 'at least once a week': '0-intensive', 'once a month':'1-moderate', 'twice a month': '1-moderate', 'less than once a month': '2-low' })
df=df.drop(df[df['V4']=="don't know"].index)
df=df.drop(df[df['V4']=="not filled in"].index)
df['V16'].value_counts()

nee              1921
ja, misschien     521
ja, zeker         457
weet niet         320
niet ingevuld       6
Name: V16, dtype: int64

In [34]:
df['V8'].replace('not filled in', 'no', inplace=True)
df['V8'] = pd.get_dummies(df['V8'])
# df['V8'].value_counts()

In [35]:
df['V5'] = df['V5'].map({'fairly': 2, 'a lot': 3, 'a little': 1, 'not': 0, 'not filled in': 0})

In [36]:
df=df.drop(df[df['V12']=="not filled in"].index)
df['V12'] = pd.get_dummies(df['V12'])

In [37]:
df=df.drop(df[df['V16']=="niet ingevuld"].index)
df['V16'] = df['V16'].map({'nee': 0, 'weet niet': 0, 'ja, misschien': 1, 'ja, zeker': 2})
df['V16'].value_counts(normalize=True)

0.0    0.695747
1.0    0.162602
2.0    0.141651
Name: V16, dtype: float64

In [8]:
famdum = pd.get_dummies(df['hhcom'])

In [9]:
df['children'] = famdum['couple with child(ren) at home'] + famdum['one parent with child(ren) at home'] + famdum['two adults, with child(ren) at home'] + famdum['three or more adults, with child(ren) at home']

In [10]:
df['couple'] = famdum['couple without children (at home)'] + famdum['couple with child(ren) at home']

In [11]:
df['houseshare'] = famdum['two adults, without children (at home)'] + famdum['two adults, with child(ren) at home'] + famdum['three or more adults, with child(ren) at home'] + famdum['three or more adults, without children at home']

In [12]:
df['V76a'].value_counts()

The Netherlands                                                                     2267
Europe (including former Sovjet-Republics)                                           150
Morocco                                                                              139
Surinam                                                                              138
Turkey                                                                               132
remaining Asia, namely                                                                83
unknown (no answer)                                                                   75
US, Canada, Australia, New Zealand, remaining Oceania, Japan, Indonesia, The Dut      65
remaining Middle- and South Amerika, namely                                           48
The Antillies, Aruba                                                                  42
remaining Africa, namely                                                              35
Ghana                

In [13]:
df=df.drop(df[df['V76a']=="unknown (no answer)"].index)
df['V76a'] = df['V76a'].map({'The Netherlands': 'Dutch Native', 'Europe (including former Sovjet-Republics)': 'Europe', 'Ghana': 'Africa', 'remaining Africa, namely': 'Africa', 'Surinam': 'Central and South America', 'The Antillies, Aruba': 'Central and South America', 'remaining Asia, namely':'Asia', 'US, Canada, Australia, New Zealand, remaining Oceania, Japan, Indonesia, The Dut':'Others', 'remaining Middle- and South Amerika, namely':'Central and South America'})

In [14]:
countrydum = pd.get_dummies(df['V76a'])
df['V76a'].value_counts(normalize=True)

Dutch Native                 0.794881
Central and South America    0.079944
Europe                       0.052595
Asia                         0.029102
Others                       0.022791
Africa                       0.020687
Name: V76a, dtype: float64

In [15]:
df.dropna(subset=['V4'], inplace=True)
df_X = pd.DataFrame()
df_X[['Trust in Neighbours', 'Neighbourhood Safety', 'House Owner', 'Plans to move']]=df[['V5', 'V8', 'V12', 'V16']]
df_X[['Africa', 'Asia', 'Central and South America', 'Dutch Native', 'Europe', 'Others']] = countrydum
df_X[['children', 'couple', 'houseshare']] = df[['children', 'couple', 'houseshare']]
df_X.shape

(3069, 13)

In [16]:
Y = df['V4']
X = sm.add_constant(df_X, prepend = False)
X['Trust in Neighbours'].value_counts()

2    1722
3     795
1     322
0     230
Name: Trust in Neighbours, dtype: int64

In [17]:
model=sm.MNLogit(Y, X.astype(float), missing='raise')
model_fit = model.fit()

Optimization terminated successfully.
         Current function value: 1.011183
         Iterations 5


In [18]:
print(model_fit.get_margeff().summary()) 

       MNLogit Marginal Effects      
Dep. Variable:                     V4
Method:                          dydx
At:                           overall
           V4=0-intensive      dy/dx    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Trust in Neighbours           0.1134      0.011     10.004      0.000       0.091       0.136
Neighbourhood Safety         -0.0290      0.018     -1.610      0.107      -0.064       0.006
House Owner                  -0.0303      0.017     -1.796      0.072      -0.063       0.003
Plans to move                -0.0458      0.012     -3.836      0.000      -0.069      -0.022
Africa                       -0.0519      0.068     -0.764      0.445      -0.185       0.081
Asia                         -0.0607      0.058     -1.051      0.293      -0.174       0.052
Central and South America    -0.0294      0.041     -0.715      0.475      -0.110       0.051
Du

In [19]:
print(model_fit.summary())

                          MNLogit Regression Results                          
Dep. Variable:                     V4   No. Observations:                 3069
Model:                        MNLogit   Df Residuals:                     3041
Method:                           MLE   Df Model:                           26
Date:                Mon, 21 Sep 2020   Pseudo R-squ.:                 0.06400
Time:                        17:24:46   Log-Likelihood:                -3103.3
converged:                       True   LL-Null:                       -3315.5
Covariance Type:            nonrobust   LLR p-value:                 1.286e-73
            V4=1-moderate       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Trust in Neighbours          -0.2254      0.074     -3.067      0.002      -0.369      -0.081
Neighbourhood Safety          0.2074      0.109      1.908      0.056      -0.006     

In [24]:
X.sum()


Trust in Neighbours          6151.0
Neighbourhood Safety          988.0
House Owner                  1478.0
Plans to move                1353.0
Africa                         54.0
Asia                           82.0
Central and South America     223.0
Dutch Native                 2242.0
Europe                        147.0
Others                         65.0
children                     1063.0
couple                       1443.0
houseshare                    603.0
const                        3069.0
dtype: float64